In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_excel("/content/recenzje_z_sentymentem.xlsx")

In [ ]:
data.head()

,recenzja,liczba gwiazdek,sentyment,adres URL
0,"Historia cudna, główni bohaterowie francuskiej...",10,1,https://www.filmweb.pl/film/Nietykalni-2011-58...
1,To dobry film i daje dużo do myślenia. W sumie...,8,1,https://www.filmweb.pl/film/Dobrze+się+kłamie+...
2,Dobry film.Jak na komedię to moim zdaniem jest...,7,1,https://www.filmweb.pl/film/Dobrze+się+kłamie+...
3,"Nigdy nie oceniałem filmu na filmwebie, jakoś ...",10,1,https://www.filmweb.pl/film/Narodziny+gwiazdy-...
4,"Film bardzo mi się spodobał, jednak zastanawia...",7,1,https://www.filmweb.pl/film/Eksperyment-2010-4...


In [ ]:
data['sentyment'].value_counts()

 1    400
-1    300
 0    300
Name: sentyment, dtype: int64

In [ ]:
def rescale_sentiment(x):
  if x == 1:
    return x
  elif x == 0:
    return 2
  elif x == -1:
    return 0

In [ ]:
data['sentyment'] = data['sentyment'].map(rescale_sentiment)

In [ ]:
data['sentyment'].value_counts()

1    400
2    300
0    300
Name: sentyment, dtype: int64

In [ ]:
with open('/content/stopwords.txt', "r") as f:
  STOP_WORDS = set([line.rstrip("\n") for line in f])

In [ ]:
import re

def preprocess(intext: str):
  text = re.sub(r'\W+', ' ', intext.lower()).split()
  text = " ".join([word for word in text if word not in STOP_WORDS])

  return text


In [ ]:
data['recenzja'] = data['recenzja'].map(preprocess)

In [ ]:
data.head()

,recenzja,liczba gwiazdek,sentyment,adres URL
0,historia cudna główni bohaterowie francuskiej ...,10,1,https://www.filmweb.pl/film/Nietykalni-2011-58...
1,dobry film daje myślenia sumie ciężki widza ni...,8,1,https://www.filmweb.pl/film/Dobrze+się+kłamie+...
2,dobry film komedię zdaniem zabawny film ciekaw...,7,1,https://www.filmweb.pl/film/Dobrze+się+kłamie+...
3,oceniałem filmu filmwebie kręciło założyłem ko...,10,1,https://www.filmweb.pl/film/Narodziny+gwiazdy-...
4,film spodobał zastanawia kwestii pierwsze stra...,7,1,https://www.filmweb.pl/film/Eksperyment-2010-4...


In [ ]:
film_data = data

In [ ]:
film_data.to_csv('/content/drive/My Drive/filmweb_preprocessed.csv')

In [ ]:
!pip install transformers==2.8.0 -q

     |████████████████████████████████| 573kB 8.1MB/s 
     |████████████████████████████████| 890kB 16.9MB/s 
     |████████████████████████████████| 3.7MB 31.3MB/s 
     |████████████████████████████████| 1.1MB 58.8MB/s 
     |████████████████████████████████| 133kB 54.4MB/s 
     |████████████████████████████████| 6.9MB 55.4MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
ERROR: botocore 1.19.25 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
from transformers import *
import torch

In [ ]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

#model.cuda()
device = torch.device("cuda")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
film_data = pd.read_csv('/content/drive/My Drive/filmweb_preprocessed.csv')

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

X = film_data.recenzja.to_numpy()
y = film_data.sentyment.to_numpy()
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in sss.split(X, y):
  train_sentences, test_sentences = X[train_index], X[test_index]
  train_labels, test_labels = y[train_index], y[test_index]

X = test_sentences
y = test_labels
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.33, random_state=42)
for train_index, test_index in sss.split(X, y):
  dev_sentences, test_sentences = X[train_index], X[test_index]
  dev_labels, test_labels = y[train_index], y[test_index]

In [ ]:
print('Train data')
print(len(train_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))
print('Dev data')
print(len(dev_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))
print('Test data')
print(len(test_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))

Train data
0.7
Dev data
0.201
Test data
0.099


In [ ]:
dev_filmweb_data = pd.DataFrame([dev_sentences,dev_labels]).T
test_filmweb_data = pd.DataFrame([test_sentences,test_labels]).T

In [ ]:
dev_filmweb_data.to_csv('/content/drive/My Drive/dev_filmweb_data_preprocessed.csv')
test_filmweb_data.to_csv('/content/drive/My Drive/test_filmweb_data_preprocessed.csv')

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

X = train_sentences
y = train_labels
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X, y):
  train_sentences, test_sentences = X[train_index], X[test_index]
  train_labels, test_labels = y[train_index], y[test_index]

X = test_sentences
y = test_labels
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for train_index, test_index in sss.split(X, y):
  dev_sentences, test_sentences = X[train_index], X[test_index]
  dev_labels, test_labels = y[train_index], y[test_index]

In [ ]:
print('Train data')
print(len(train_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))
print('Dev data')
print(len(dev_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))
print('Test data')
print(len(test_sentences) / ( len(train_sentences) + len(dev_sentences) + len(test_sentences) ))

Train data
0.8
Dev data
0.1
Test data
0.1


In [ ]:
# Remove long sentences.
# TO-DO Possible cut?
def remove_big(sentences, labels):
  to_remove = []
  for i, sent in enumerate(sentences):
      input_ids = tokenizer.encode(sent, add_special_tokens=True) # TO-DO: add_special_tokens
      if len(input_ids) > MAX_LEN:
        to_remove.append(i)

  sentences = np.delete(sentences, to_remove)
  labels = np.delete(labels, to_remove) 

  print('{} samples removed.'.format(len(to_remove)))

  return sentences, labels

In [ ]:
# Downloading tokenizer
# From Polbert - Polish BERT by Darek Kłeczek: https://github.com/kldarek/polbert
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

In [ ]:
MAX_LEN = 128

train_sentences, train_labels = remove_big(train_sentences, train_labels)
test_sentences, test_labels = remove_big(test_sentences, test_labels)
dev_sentences, dev_labels = remove_big(dev_sentences, dev_labels)

0 samples removed.
1 samples removed.
0 samples removed.


In [ ]:
from torch.utils.data import TensorDataset
# Create TensorDatasets for train/dev/test sets
def tensor_dataset(sentences, labels):
  input_ids = []
  attention_masks = []

  for sent in sentences:
      encoded_dict = tokenizer.encode_plus(
                          sent,                     
                          add_special_tokens = True,
                          max_length = MAX_LEN,
                          pad_to_max_length = True,
                          return_attention_mask = True,
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)
  dataset = TensorDataset(input_ids, attention_masks, labels)
  return dataset

In [ ]:
BATCH_SIZE = 8

train_dataset = tensor_dataset(train_sentences, train_labels)
test_dataset = tensor_dataset(test_sentences, test_labels)
dev_dataset = tensor_dataset(dev_sentences, dev_labels)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# Create the DataLoaders for train/dev/test sets.
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = BATCH_SIZE)
validation_dataloader = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = BATCH_SIZE)

In [ ]:
# Load model with a sequence classification head
model = BertForSequenceClassification.from_pretrained(
    "dkleczek/bert-base-polish-uncased-v1", # Polbert - Polish BERT by Darek Kłeczek: https://github.com/kldarek/polbert
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(60000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import time, datetime
import numpy as np
from tqdm import tqdm
from transformers.optimization import AdamW
from transformers import AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Takes a time in seconds and returns a string hh:mm:ss
def format_time(elapsed):
  elapsed_rounded = int(round((elapsed)))
  return str(datetime.timedelta(seconds=elapsed_rounded))

# Parameters:
epochs = 3
#lr = 3e-3 # Learning rate (Adam): 5e-5, 3e-5, 2e-5
lr = 5e-5 # Learning rate (Adam): 5e-5, 3e-5, 2e-5
adam_epsilon = 1e-10
WARM_UP = 0

optimizer = AdamW(model.parameters(), lr = lr, eps = adam_epsilon)

from transformers import get_linear_schedule_with_warmup
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = WARM_UP, num_training_steps = total_steps)

train_loss_values = []
dev_acc_values = []

model.zero_grad()

t0 = time.time()
for epoch_i in range(0, epochs):  
  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  # https://github.com/huggingface/transformers/blob/master/examples/run_glue.py
  # linie 168-183
  epoch_train_loss = 0 # Cumulative loss
  loss = 0 ;     batch_loss = 0
  model.train()

  for step, batch in enumerate(train_dataloader):

    # Progress update every 40 batches.
    if step % 40 == 0 and not step == 0:
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)      
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}. Loss: {:.3f}  Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))
    
    batch_loss = 0
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)    

    # clear any previously calculated gradients before backward pass
    optimizer.zero_grad()

    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

    loss = outputs[0]
    epoch_train_loss += loss.item()
    batch_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
    optimizer.step()
    scheduler.step()  # Update learning rate schedule

  epoch_train_loss = epoch_train_loss / len(train_dataloader)          
  train_loss_values.append(epoch_train_loss)
  
  print('Average training loss: {0:.2f}'.format(epoch_train_loss))

  # Evaluation
  total_eval_accuracy = 0
  model.eval()

  for batch in validation_dataloader:
    
    input_ids = batch[0].to(device)
    attention_masks = batch[1].to(device)
    labels = batch[2].to('cpu').numpy()
                
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()

    predictions = np.argmax(logits, axis=1).flatten()
    total_eval_accuracy += flat_accuracy(logits, labels)

  avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
  print("  Accuracy: {0:.4f}".format(avg_val_accuracy))



======== Epoch 1 / 3 ========
Training...


/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


  Batch    40  of     70. Loss: 1.121  Elapsed: 0:00:08.
Average training loss: 0.96
  Accuracy: 0.5787

======== Epoch 2 / 3 ========
Training...
  Batch    40  of     70. Loss: 0.181  Elapsed: 0:00:23.
Average training loss: 0.45
  Accuracy: 0.5880

======== Epoch 3 / 3 ========
Training...
  Batch    40  of     70. Loss: 0.520  Elapsed: 0:00:38.
Average training loss: 0.15
  Accuracy: 0.6481


In [ ]:
predicted_labels = [] ; true_labels = []; logits_list = []

for batch in test_dataloader:
  
  input_ids = batch[0].to(device)
  attention_masks = batch[1].to(device)
  labels = batch[2]
  
  with torch.no_grad():        
      outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
                  
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  logits_list.append(logits)
  
  predictions = np.argmax(logits, axis=1).flatten()
  labels = labels.numpy().flatten()

  predicted_labels.extend( predictions )
  true_labels.extend( labels )
  

In [ ]:
# Parameters:
#epochs = 3
#lr = 2e-5 # Learning rate (Adam): 5e-5, 3e-5, 2e-5
#adam_epsilon = 1e-8
#WARM_UP = 0.1
#0.72-73
from sklearn.metrics import classification_report 
print( classification_report(y_true=true_labels, y_pred=predicted_labels, zero_division=0) )

              precision    recall  f1-score   support

           0       0.58      0.75      0.65        20
           1       0.81      0.79      0.80        28
           2       0.56      0.43      0.49        21

    accuracy                           0.67        69
   macro avg       0.65      0.65      0.65        69
weighted avg       0.67      0.67      0.66        69



In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from transformers import WEIGHTS_NAME, CONFIG_NAME
output_dir = "/content/drive/My Drive/model_bert_finetuned_3"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned
# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)

model_to_save = model.module if hasattr(model, 'module') else model
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_pretrained(output_dir)

('/content/drive/My Drive/model_bert_finetuned_3/vocab.txt',
 '/content/drive/My Drive/model_bert_finetuned_3/special_tokens_map.json',
 '/content/drive/My Drive/model_bert_finetuned_3/added_tokens.json')

In [ ]:
# Step 2: Re-load the saved model and vocabulary
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)